In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import os
import random
from geopy.distance import geodesic
from plotly import express as px
import json 
import plotly.graph_objects as go
import geopandas as gpd


## Plotting helper functions

In [2]:
# Function to extract coordinates from GeoJSON and draw boundaries
def add_geojson_boundaries(fig, geojson):
    for feature in geojson['features']:
        geometry = feature['geometry']
        coords = []
        
        if geometry['type'] == 'Polygon':
            coords = geometry['coordinates'][0]  # Outer boundary
        elif geometry['type'] == 'MultiPolygon':
            for polygon in geometry['coordinates']:
                coords.extend(polygon[0])
                coords.append([None, None])  # Separator for discontinuous lines
        
        
        if coords:
            lons, lats = zip(*coords)
            fig.add_trace(go.Scattergeo(
                lon=lons,
                lat=lats,
                mode='lines',
                line=dict(width=1, color='lightblue'),
                showlegend=False,
                hoverinfo='skip'
            ))


# National population (by state)

In [ ]:
# downloaded from : https://www.inegi.org.mx/sistemas/Olap/Proyectos/bd/censos/cpv2020/PHV.asp?
# currently missing aguascalientes
mexico_states = {
    # "Aguascalientes": "01", # TODO: find write EXCEL file for Aguascalientes
    "Baja California": "02", "Baja California Sur": "03","Campeche": "04", "Coahuila de Zaragoza": "05","Colima": "06", "Chiapas": "07","Chihuahua": "08","Ciudad de México": "09","Durango": "10","Guanajuato": "11","Guerrero": "12","Hidalgo": "13","Jalisco": "14","México": "15", "Michoacán de Ocampo": "16", "Morelos": "17", "Nayarit": "18", "Nuevo León": "19", "Oaxaca": "20","Puebla": "21", "Querétaro": "22", "Quintana Roo": "23", "San Luis Potosí": "24", "Sinaloa": "25", "Sonora": "26", "Tabasco": "27", "Tamaulipas": "28", "Tlaxcala": "29", "Veracruz de Ignacio de la Llave": "30", "Yucatán": "31", "Zacatecas": "32"
}



one_state = False
first_rows = []
if one_state:
    state = 'San Luis Potosí'
    # these excel files are not included in the repository
    filename = f'../Datos Nacionales/POBLACION/ITER_{mexico_states[state]}XLSX20.xlsx'
    pop_df = pd.read_excel(filename)

else:
    # again, not included in repo
    gdf = gpd.read_file('../Datos SLP/shapefiles/mexico_by_estado.json')
    # gdf = gdf.to_crs(6372)
    gdf['centroid'] = gdf['geometry'].centroid
    
    gdf = gdf.set_index('name', drop='False')
    pop_df = pd.read_excel('../Datos Nacionales/POBLACION/pop_by_state.xlsx')



# Combined contaminant data

In [ ]:
# data not included, but downloaded from DENUE, cleaned by Lizet Jarquin and Jacqueline Calderon
contaminants_xl = pd.ExcelFile('../Datos Nacionales/BaseNacional-Ladr-Agua-Mina.xlsx')
cont_ladr = pd.read_excel(contaminants_xl,'Ladrilleras')
cont_ladr = cont_ladr.rename(columns={'x': 'X', 'y':'Y', 'municipio': 'MUNICIPIO'})
cont_ladr['tipo'] = 'ladrillera'

# mining and water data not related to today's challenge
cont_minas = pd.read_excel(contaminants_xl,'Minas')
cont_minas['tipo'] = 'mina'
cont_minas = cont_minas.rename(columns={'Municipio': 'MUNICIPIO'})
cont_agua = pd.read_excel(contaminants_xl,'Agua')
cont_agua['tipo'] = 'agua'
cols_to_replace = ['Acrilonitrilo', 'Benceno', 'Arsenico_Total', 'Cadmio_Total', 'Arsenico_Soluble', 'Cadmio_Soluble','metales', 'elementos']


contaminants_all = pd.concat([cont_ladr, cont_agua, cont_minas], ignore_index=True)
contaminants_all[cols_to_replace] = contaminants_all[cols_to_replace].fillna(0)


In [22]:
# downlaoded from https://github.com/PhantomInsights/mexico-geojson/blob/main/2023/%60mexico.zip
with open('../Datos SLP/shapefiles/mexico_by_estado.json', 'r') as f:
    mex_geojson = json.load(f)

In [40]:
# point sources of ladrilleras
# Create figure
fig = go.Figure()

# Add GeoJSON boundaries
add_geojson_boundaries(fig, mex_geojson)

color_col = 'tipo'
# Add scatter points
for eff in cont_ladr[color_col].unique():
    mun_data = cont_ladr[cont_ladr[color_col] == eff]
    # print(mun_data)
    fig.add_trace(go.Scattergeo(
        lon=mun_data['X'],
        lat=mun_data['Y'],
        text=[mun_data['MUNICIPIO']],
        name=eff,
        mode='markers',
        marker=dict(size=8), 
        opacity=0.5
    ))

fig.update_geos(
    fitbounds="locations",
    visible=True
)

fig.update_layout(
    height=600,
    showlegend=True,
    title_text='Mapa de ladrilleras'
)

fig.show()
fig.write_html('map_ladrilleras_point_sources.html')

# Computing effects

In [53]:
# assigns a point-score that's inverse proportional to the distance to a ladrillera; will be summed over all ladrilleras
def pollutant_effect(pollutant, location,  lon_name = 'X', lat_name = 'Y'):
    man_loc = (location[lat_name], location[lon_name])
    
    pol_loc = (pollutant[lat_name], pollutant[lon_name])
    
    distance = geodesic(man_loc, pol_loc).kilometers
    
        
    # TODO: revisit if we want to make this more complex
    return 1. / distance


pollutant_list = ['Ladrilleras']

effects_df = pd.DataFrame()
effects_df['NOM_ENT'] = pop_df['NOM_ENT']


for pol in pollutant_list:
    effects_df[pol] = 0.0


for pol in pollutant_list:
    idx = 0
    for i, location in pop_df.iterrows():
        effect_counter = 0


        for j, measurement in cont_ladr.iterrows():
            

            effect = pollutant_effect(measurement, location, pol_name = pol)
            effect_counter += effect
        
        effects_df.loc[i, pol] = effect_counter
        # print(f'Effect of {pol} on {location["NOM_ENT"]} is {effect_counter}')
        idx += 1

# print(effects_df.head())

# Heatmap of effects of ladrilleras by state

In [54]:

print(effects_df.head())
fig = go.Figure()
pollutant = 'Ladrilleras'
fig = px.choropleth(
    effects_df, 
    geojson=mex_geojson, 
    color=pollutant, 
    locations='NOM_ENT',  # column in state_pop_df that matches GeoJSON features
    featureidkey='properties.name',  # key in GeoJSON properties
    color_continuous_scale='Greens'
)
fig.update_geos(
    fitbounds="locations"
)
fig.update_layout(
    height=600,
    showlegend=True, title_text=f'Effect of {pollutant}, not adjusted for population'
)
fig.show()
# fig.write_html('ladrilleras_by_state.html')

In [ ]:
from shapely.geometry import box
# Get the bounds of your data
# Read your GeoJSON file


# Get the bounds of your data
minx, miny, maxx, maxy = gdf.total_bounds

# Important: Check if your data is in geographic coordinates (lat/lon)
if gdf.crs and gdf.crs.is_geographic:
    # Reproject to a projected CRS for accurate distance measurements
    # Use an appropriate UTM zone or equal-area projection
    # This example uses Web Mercator, but UTM is better for accuracy
    gdf_projected = gdf.to_crs('EPSG:6372')  # Web Mercator
    minx, miny, maxx, maxy = gdf_projected.total_bounds
    original_crs = gdf.crs
else:
    gdf_projected = gdf
    original_crs = gdf.crs

grid_size = 50000  # 50 km in meters

# Generate grid cell coordinates
x_coords = np.arange(minx, maxx, grid_size)
y_coords = np.arange(miny, maxy, grid_size)

# Create grid polygons
grid_cells = []
for x in x_coords:
    for y in y_coords:
        grid_cells.append(box(x, y, x + grid_size, y + grid_size))

# Create GeoDataFrame from grid
grid_gdf = gpd.GeoDataFrame(geometry=grid_cells, crs=gdf_projected.crs)

# Reproject back to original CRS if needed
if original_crs:
    grid_gdf = grid_gdf.to_crs(original_crs)

# Optional: Only keep grid cells that intersect with your original data
grid_gdf = grid_gdf[grid_gdf.intersects(gdf.unary_union)]


if grid_gdf.crs != 'EPSG:4326':
    grid_gdf = grid_gdf.to_crs('EPSG:4326')




# Get centroids of grid cells
grid_gdf['centroid'] = grid_gdf.geometry.centroid
grid_gdf['X'] = grid_gdf['centroid'].x
grid_gdf['Y'] = grid_gdf['centroid'].y
print(len(grid_gdf))
print(grid_gdf.iloc[0])

In [ ]:
def pollutant_effect_grid(pollutant, location, lon_name = 'X', lat_name = 'Y'):
    man_loc = (location[lat_name], location[lon_name])
    pol_loc = (pollutant[lat_name], pollutant[lon_name])
    
    distance = geodesic(man_loc, pol_loc).kilometers       
    
    # TODO: revisit if we want to make this more complex
    return 1. / distance

pollutant_list = ['Ladrilleras']


effects_df = pd.DataFrame()


for pol in pollutant_list:
    effects_df[pol] = 0.0


for pol in pollutant_list:
    idx = 0
    for i, location in grid_gdf.iterrows():
        effect_counter = 0


        for j, measurement in cont_ladr.iterrows():
            effect = pollutant_effect_grid(measurement, location, pol_name = pol)
            effect_counter += effect
        
        effects_df.loc[i, pol] = effect_counter
        if idx % 50 == 0:
            print(f'Effect of {pol} on {idx} is {effect_counter}')
        idx += 1
        
# WARNING: TAKES 10 MINUTES TO RUN AT 50KM RESOLUTION
print(effects_df.head())

In [ ]:
grid_gdf['Ladrilleras'] = effects_df['Ladrilleras']
grid_gdf['log_column'] = np.log10(grid_gdf['Ladrilleras'])


# Add GeoJSON boundaries
add_geojson_boundaries(fig, mex_geojson)
# Create scatter_geo plot
fig = px.scatter_geo(
    grid_gdf,
    lat='Y',
    lon='X',
    color='log_column',
    color_continuous_scale='Hot',
    hover_data=['Ladrilleras'],  # Optional: show coordinates on hover,
    labels={'log_column': 'log10(Ladrilleras score)'}
)

fig.update_traces(marker=dict(size=6, opacity=0.9))
fig.update_geos(
    fitbounds="locations"
)

color_col = 'tipo'
# Add scatter points
for eff in cont_ladr[color_col].unique():
    mun_data = cont_ladr[cont_ladr[color_col] == eff]
    fig.add_trace(go.Scattergeo(
        lon=mun_data['X'],
        lat=mun_data['Y'],
        text=[mun_data['MUNICIPIO']],
        name=eff,
        mode='markers',
        marker=dict(size=5), 
        opacity=0.2
    ))

fig.update_layout(
    height=600,
    showlegend=False, title_text=f'Effect of ladrilleras on grid'
)

fig.show()
fig.write_html('ladrilleras_grid_overlaid.html')